 # Lightweight Multi-Branch Network for Animal Individual Re-Identification

Check if you are using GPU

*If not, go to Edit -> "Notebook settings" or Runtime -> "Change runtime type" and select GPU as hardware accelerator.*

Note: Tesla T4 is faster than Tesla K80.

In [ ]:
!nvidia-smi

Download code

In [ ]:
%%capture
!gdown --id 11Z3PlDTlPXV_BpOHA0_8ChLh_g2O8T6H
!unzip LightMBN.zip
!rm LightMBN.zip

### Train on BayWald Dataset
Note: If label smoothing is enabled, CrossEntropy cannot fall below 2.0.

In [ ]:
!python /content/LightMBN/main.py \
        --data_train      baywald \
        --data_test       baywald \
        --height          180 \
        --width           320 \
        --epochs          30 \
        --use_masks       \
        --frame_dropping  \
        --query_gallery_separation \
        --test_every      1 \
        --test_pretrained \
        --batchid         3 \
        --batchimage      10 \
        --batchtest       30 \
        --model           LMBN_n \
        --feats           512 \
        --loss            0.5*CrossEntropy+0.5*MSLoss \
        --lr              2e-4 \
        --if_labelsmooth  \
        --w_cosine_annealing \
        --optimizer       ADAM \
        --margin          0.7 \
        --datadir         /content \
        --nGPU            1 \
        --nThread         2 \
        --save            "baywald"

## Train on Wildpark Dataset

In [ ]:
!python /content/LightMBN/main.py \
        --data_train      wildpark \
        --data_test       wildpark \
        --height          180 \
        --width           240 \
        --epochs          20 \
        --frame_dropping  \
        --query_gallery_separation \
        --test_every      1 \
        --test_pretrained \
        --batchid         3 \
        --batchimage      10 \
        --batchtest       30 \
        --model           LMBN_n \
        --feats           512 \
        --loss            0.5*CrossEntropy+0.5*MSLoss \
        --lr              1.5e-4 \
        --if_labelsmooth  \
        --w_cosine_annealing \
        --optimizer       ADAM \
        --margin          0.7 \
        --datadir         /content \
        --nGPU            1 \
        --nThread         2 \
        --save            "wildpark"

Visualize activation maps

In [ ]:
dir = "baywald" #baywald/wildpark/market1501
!python /content/LightMBN/utils/visualize_actmap.py \
        --activation_map \
        --config "/content/LightMBN/experiment/$dir/config.yaml" \
        --save $dir

Visualize rank

In [ ]:
!python /content/LightMBN/utils/visualize_rank.py \
        --config "/content/LightMBN/experiment/$dir/config.yaml" \
        --save $dir

Make a graph manually

In [4]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch
import numpy as np

log = torch.Tensor([
    [1, 7.146321, 1.291028, 8.437350],
    [2, 3.992681, 0.697871, 4.690553],
    [3, 2.516816, 0.522015, 3.038830],
    [4, 1.814123, 0.343739, 2.157861],
    [5, 1.404565, 0.692838, 2.097403],
    [6, 1.136525, 0.401017, 1.537541],
    [7, 0.819759, 0.609123, 1.428882],
    [8, 0.684335, 0.225319, 0.909653],
    [9, 0.553456, 0.580251, 1.133707],
    [10, 0.464284, 0.555549, 1.019833],
    [11, 0.367480, 0.568028, 0.935508],
    [12, 0.311656, 0.626420, 0.938076],
    [13, 0.260951, 0.519575, 0.780527],
    [14, 0.256252, 0.408198, 0.664450],
    [15, 0.186960, 0.755801, 0.942761],
    [16, 0.166804, 0.652810, 0.819614],
    [17, 0.158289, 0.717179, 0.875469],
    [18, 0.136102, 0.682132, 0.818234],
    [19, 0.111095, 0.658287, 0.769382],
    [20, 0.095065, 0.829413, 0.924478],
    [21, 0.095022, 0.684957, 0.779979],
    [22, 0.080752, 0.795032, 0.875784],
    [23, 0.083969, 0.591880, 0.675849],
    [24, 0.074384, 0.617269, 0.691654],
    [25, 0.070003, 0.714642, 0.784645],
    [26, 0.061606, 0.495032, 0.556638],
    [27, 0.060110, 0.592547, 0.652656],
    [28, 0.047353, 0.491587, 0.538940],
    [29, 0.054237, 0.358431, 0.412668],
    [30, 0.050482, 0.526776, 0.577258],
])

def plot_map_rank():
    axis = np.linspace(1, 30, log.size(0))
    label = 'Reid on baywald'
    labels = ['CrossEntropy', 'MSLoss', 'Total']
    fig = plt.figure()
    plt.title(label)
    for i in range(len(labels)):
        plt.plot(axis, log[:, i + 1].numpy(), label=labels[i])
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('loss.pdf', dpi=600)
    plt.close(fig)

plot_map_rank()

Download results to local machine

In [ ]:
from google.colab import files

!zip -r /content/result.zip /content/LightMBN/experiment/$dir
files.download("/content/result.zip")

## **Reproduce PREID results**

Train on Market1501 Dataset

In [ ]:
!python /content/LightMBN/main.py \
        --data_train      market1501 \
        --data_test       market1501 \
        --epochs          110 \
        --test_every      20 \
        --test_pretrained \
        --batchid         6 \
        --batchimage      8 \
        --batchtest       32 \
        --model           LMBN_n \
        --feats           512 \
        --loss            0.5*CrossEntropy+0.5*MSLoss \
        --lr              6e-4 \
        --random_erasing  \
        --if_labelsmooth  \
        --w_cosine_annealing \
        --optimizer       ADAM \
        --margin          0.7 \
        --datadir         /content \
        --nGPU            1 \
        --nThread         2 \
        --save            "market1501"

Test a pretrained model on CUHK03 Dataset

In [ ]:
import gdown

gdown.download("https://drive.google.com/uc?export=download&id=1BeXrNluAvaLsNvXyEK0WFHM6kDw3r8ND", "PRETRAIN.zip", True)
!unzip PRETRAIN.zip
!rm PRETRAIN.zip

In [ ]:
!python /content/LightMBN/main.py \
        --test_only
        --config      "/content/PRETRAIN/cfg_lmbn_n_cuhk03_d.yaml" \
        --pre_train   "/content/PRETRAIN/lmbn_n_cuhk03_d.pth"